In [1]:
import glob, pandas as pd, numpy as np, re
# import nltk
# nltk.download('all')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from tqdm import tqdm

sws = stopwords.words("english") + ["n't",  "'s", "'ve"]

In [5]:
fls = glob.glob("../../data/feedback-effectiveness/train/*.txt")
docs = []
for fl in tqdm(fls):
    with open(fl) as f:
        txt = f.read()
        word_tokens = word_tokenize(txt)
        txt = " ".join([w for w in word_tokens if not w.lower() in sws])
    docs.append(str(txt))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 4191/4191 [00:07<00:00, 568.77it/s]


In [6]:
len(docs)

4191

In [7]:
from bertopic import BERTopic
topic_model = BERTopic(n_gram_range=(1, 3), top_n_words=5, verbose=True)
topics, probs = topic_model.fit_transform(docs)

Batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 131/131 [01:25<00:00,  1.54it/s]
2023-03-28 20:17:00,252 - BERTopic - Transformed documents to Embeddings
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
2023-03-28 20:17:09,899 - BERTopic - Reduced dimensionality
2023-03-28 20:17:09,952 - BERTopic - Clustered reduced embeddings


In [8]:
tm_meta = topic_model.get_topic_info()
tm_meta.to_csv("topic_model_metadata.csv", index=False)
display(tm_meta)

pred_topics = pd.DataFrame()
dids = list(map(lambda fl: fl.split("/")[-1].split(".")[0], fls))
pred_topics["id"] = dids
pred_topics["topic"] = topics
pred_topics['prob'] = probs
pred_topics.to_csv("topic_model_feedback.csv", index=False)
pred_topics

,Topic,Count,Name
0,-1,16,-1_genericname_school_people_community
1,0,876,0_electoral_vote_college_electoral college
2,1,410,1_emotions_technology_computer_facial
3,2,398,2_venus_planet_earth_author
4,3,325,3_face_mars_aliens_landform
5,4,303,4_students_online_school_classes
6,5,292,5_community_service_community service_help
7,6,238,6_students_summer_project_projects
8,7,209,7_advice_people_multiple_opinions
9,8,204,8_cars_driverless_car_driverless cars


,id,topic,prob
0,2E4AFCD3987F,0,1.000000
1,A91A08E523D5,0,1.000000
2,4E078101C9DD,6,0.933356
3,F9368F6BAB62,13,1.000000
4,B0CCFA21F82C,11,0.694798
...,...,...,...
4186,D91B2369C76D,5,1.000000
4187,ACCD71550365,7,1.000000
4188,4453444AF383,0,1.000000
4189,EF0D75BF48DA,0,1.000000
